IMPORTING LIBRARIES 📚

In [84]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

Create DataFrame 👾

In [85]:
transfer_table = pd.DataFrame({"player_id" : [], "season" : [], "date" : [], "left" : [], "joined" : [], "mv" : [], "fee" : []})
transfer_table = transfer_table.astype(str)
links = pd.read_csv("All_players_links.csv")

Getting Transfer Table 💻

In [86]:
def players_transfer_table(link):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36", "accept-language": "en-US,en;q=0.9"}        
    url = link
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")

    #Getting Transfer Data
    try:
        transfer_table = soup.find("div", {"data-viewport" : "Transferhistorie"})
        rows = transfer_table.find_all("div", class_ = "grid tm-player-transfer-history-grid")
        transfer_table_list = []
        for i in rows:
            clean_text = i.text.strip()
            lines = clean_text.split("\n")
            lines = [line.strip() for line in lines if line.strip()]
            if ("2017" in lines[1]) or ("2018" in lines[1]) or ("2019" in lines[1]) or ("2020" in lines[1]) or ("2021" in lines[1]):
                transfer_table_list += [lines]
        return transfer_table_list
    
    except AttributeError:
        pass

Getting Data 👩‍💻👨‍💻

In [88]:
for i in range(len(links)):
    res = {}
    #Getting Players ID
    pattern = r"/(\d+)$"
    match = re.search(pattern, links.loc[i, "0"])
    player_id = match.group(1)
    table_list = players_transfer_table(links.loc[i, "0"])
    for j in table_list:
        if len(j) == 6:
            res["player_id"] = player_id
            res["season"] = j[0]
            res["date"] = j[1]
            res["left"] = j[2]
            res["joined"] = j[3]
            res["mv"] = j[4]
            res["fee"] = j[5]
            transfer_table = transfer_table.append(res, ignore_index = True)

transfer_table = transfer_table.drop_duplicates()
#pd.DataFrame(transfer_table).to_csv("All_players_transfers.csv", index = False)